### <h2>0. Thêm Thư Viện</h2>

In [1]:
# thư viện
import numpy as np
import pandas as pd
import csv

from sklearn.model_selection import train_test_split # chia ác bộ để test and train
from sklearn.preprocessing import StandardScaler # kiểm tra dữ liệu (chuẩn hóa)
from sklearn.neighbors import KNeighborsClassifier # công cụ thuật toán
from sklearn.linear_model import RidgeClassifier # công cụ thuật toán
from sklearn.linear_model import LogisticRegression
### công cụ kiểm tra độ predict 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score


### <h2>1. Read Data</h2>

In [2]:
# read data
data1 = pd.read_csv("./file_csv/Train_samsung.csv")
data2 = pd.read_csv("./file_csv/Test_samsung_noclass.csv")
predict_data = data2.copy()
list_data = [data1, data2]
# display(list_data[0])

### <h2>2. Convert string to numberic</h2>

In [3]:
# chuyển đổi
for data in list_data:
    # print(data['X1'].unique())
    # print(data['X2'].unique())
    # print(data['X3'].unique())
    # print(data['X4'].unique())
    # print(data['X5'].unique())
    # print(data["X11"].unique())

    data['X3'].replace({"3+" : 3, "0" : 0, "1" : 1, "2" : 2}, inplace=True)
    data['X1'].replace({"Male" : 1, "Female" : 0}, inplace=True)
    data['X2'].replace({"Yes" : 1, "No" : 0}, inplace=True)
    data['X5'].replace({"Yes" : 1, "No" : 0}, inplace=True)
    data['X4'].replace({"Graduate" : 1, "Not Graduate" : 0}, inplace=True)
    data['X11'].replace({"Semiurban" : 0, "Rural" : 1, "Urban" : 2}, inplace=True)
# print(list_data[0]['Class'].unique())
list_data[0]['Class'].replace({"Y" : 1, "N" : 0}, inplace=True)
# display(list_data[0])
# display(list_data[1])

### <h2>3. Clean data</h2>

#### <h4>1. kiểm tra</h4>

In [4]:
# kiểm tra nan
def check():
    display(list_data[0].isna().sum())
    display(list_data[1].isna().sum())

#### <h4>2. function with fre</h4>

In [5]:
# làm sạch theo tần số
def clean_fre(df):
    cnt = df.value_counts(dropna=True)
    df.fillna(cnt.index[0], inplace=True)

#### <h4>3. function min max</h4>

In [6]:
# làm sạch theo median
def rnd(df):
    # round 
    df.fillna(round(df.mean()), inplace=True)

#### <h4>4. Clean data</h4>

In [7]:
# lấp đầy bằng median
for data in list_data:
    for i in range(len(data.columns) - 1):
        rnd(data[data.columns[i]])
# check()


### <h2>4. Chuẩn hóa dữ liệu</h2>

In [8]:
# chuẩn hóa về 0 đến 1
for data in list_data:
    for i in range(5, 9):
        data[data.columns[i]] = (data[data.columns[i]] - data[data.columns[i]].min()) / (data[data.columns[i]].max() - data[data.columns[i]].min())


### <h2>5. Train test data</h2>

In [9]:
# tạo train and test : cùng thư viện train_test_split()

X_train, X_test, y_train, y_test = train_test_split(list_data[0].iloc[:, :-1], list_data[0].iloc[:, -1], test_size=0.12, random_state=42)

In [10]:
# train and test RidgeClassifier
clf = RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True, 
                      max_iter=None, normalize=False, random_state=6933,
                      solver='auto', tol=0.001)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("accuracy: ", accuracy_score(y_test, y_pred))
print("f1 score: ", f1_score(y_test, y_pred, average='weighted'))
print("confusion matrix: ")
print(confusion_matrix(y_test, y_pred))


accuracy:  0.8305084745762712
f1 score:  0.8008153569594247
confusion matrix: 
[[ 6 10]
 [ 0 43]]


/home/vutuyen/.local/lib/python3.10/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(


In [11]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, cross_validate, KFold
logit = LogisticRegression()
standard_scaler = StandardScaler()

scoring = ['accuracy', 'f1']

pipeLine = make_pipeline(standard_scaler, logit)
kf = KFold(n_splits=5, shuffle=True, random_state=2)

cv_resutls = cross_validate(
    pipeLine, X_train, y_train, cv=kf, n_jobs=-1, scoring=scoring)

cv_resutls


{'fit_time': array([0.00653195, 0.00778365, 0.0071373 , 0.00612783, 0.00618505]),
 'score_time': array([0.00246763, 0.00241566, 0.00275445, 0.00227356, 0.00223374]),
 'test_accuracy': array([0.8045977 , 0.82758621, 0.81395349, 0.81395349, 0.81395349]),
 'test_f1': array([0.87407407, 0.88372093, 0.88732394, 0.87692308, 0.87692308])}

In [12]:
cv_resutls['test_f1'].mean()

0.8797930203629516

### <h2>6. Predict class in test_samsung_noclass: target:: class</h2>

In [13]:
# predict  test_samsung
x_test_samsung = list_data[1].copy()
y_test_samsung = clf.predict(x_test_samsung)
predict_data["Class"] = y_test_samsung
predict_data['Class'].replace({1 : "Y", 0 : "N"}, inplace=True)

In [14]:
# result
print(y_test_samsung)
file_csv = csv.writer(open("./file_csv/class_label.csv", "w"))
file_csv.writerow(["Class"])
for i in range(predict_data.shape[0]):
    file_csv.writerow(predict_data['Class'][i])
display(predict_data)

[1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 0 1 1 1 0 1 0 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1
 0 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1
 0 1 1 1 1 1 1 1 1 1 1 1]


,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,Class
0,Male,Yes,0,Graduate,No,9083,0.0,228.0,360.0,1.0,Semiurban,Y
1,Male,Yes,0,Graduate,No,4310,0.0,130.0,360.0,NaN,Semiurban,Y
2,Male,Yes,2,Graduate,No,4167,1447.0,158.0,360.0,1.0,Rural,Y
3,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
4,Male,No,0,Graduate,No,2718,0.0,70.0,360.0,1.0,Semiurban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...
118,Male,Yes,0,Graduate,NaN,3716,0.0,42.0,180.0,1.0,Rural,Y
119,Female,No,0,Graduate,No,2507,0.0,56.0,360.0,1.0,Rural,Y
120,Male,Yes,1,Graduate,No,2882,1843.0,123.0,480.0,1.0,Semiurban,Y
121,Male,Yes,2,Graduate,NaN,2500,1840.0,109.0,360.0,1.0,Urban,Y
